In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt

In [4]:
#Read in the original csv file
NY511 = pd.read_csv('../data/511_NY_Events__Beginning_2010.csv', low_memory=False)

#Data source: https://catalog.data.gov/nl/dataset/511-ny-events-beginning-2010

In [41]:
NY511.info() #2627317 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627317 entries, 0 to 2627316
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Event Type                  object 
 1   Organization Name           object 
 2   Facility Name               object 
 3   Direction                   object 
 4   City                        object 
 5   County                      object 
 6   State                       object 
 7   Create Time                 object 
 8   Close Time                  object 
 9   Event Description           object 
 10  Responding Organization Id  object 
 11  Latitude                    float64
 12  Longitude                   float64
dtypes: float64(2), object(11)
memory usage: 260.6+ MB


In [22]:
#List of counties with confirmed Brood 5 (2016) and Brood 8 (2018) sightings
NYcounties = ['Cayuga', 'Onondaga', 'Livingston','Suffolk']

In [5]:
#Actually, only going to look at 2016 since cant compare 2018 to 2019.
NYcounties = ['Suffolk']
#New DataFrame with relevant counties only
NY511_trim = NY511[(NY511['County'].isin(NYcounties))]

## Getting Relevant Traffic Incident Types


In [6]:
NY511_trim['Event Type'].unique()

array(['construction', 'roadwork', 'moving operation', 'incident',
       'pothole repair', 'disabled vehicle', 'overturned vehicle',
       'police department activity', 'misplaced tractor trailer', 'crash',
       'guard rail repairs', 'crash road closed', 'pothole repairs',
       'roving repairs', 'crash, downed pole', 'road sweeping',
       'disabled truck', 'brush fire', 'downed pole',
       'emergency construction', 'vehicle fire', 'pedestrian accident',
       'debris spill', 'disabled bus', 'overhead sign repairs',
       'missing sewer grate', 'accident', 'accident road closed',
       'truck fire', 'flooding', 'downed tree', 'accident investigation',
       'street sweeping', 'disabled tractor trailer',
       'construction, road sweeping', 'utility work', 'other',
       'accident, jack-knifed tractor trailer',
       'fire department activity', 'gas main break',
       'traffic signal repairs', 'accident, downed wires', 'fuel spill',
       'mowing', 'downed pole, downed

In [15]:
#Relevant'Event Type' values to keep
#assumptions made: These are unplanned events typically caused by human(s) action.  
road_events=['accident', 'incident', 'delays, stop and go traffic', 
'delays, earlier incident', 'debris spill', 'earlier incident', 
'accident, delays', 'delays', 'accident road closed', 'downed wires, downed pole', 'downed pole',
'accident, disabled vehicle', 'delays, stop and go traffic, traffic congestion',
'debris spill, earlier incident', 'accident, overturned tractor trailer', 'accident, emergency maintenance', 
'delays, heavy traffic', 'overturned tractor trailer', 'accident, jack-knifed tractor trailer', 
'jack-knifed tractor trailer', 'accident, overturned truck', 'overturned truck', 'stop and go traffic, traffic congestion',
'rubbernecking delays', 'accident, debris spill']


In [16]:
#trim dataframe to match the list of indivents/event types
NY511_trim = NY511_trim[(NY511_trim['Event Type'].isin(road_events))]

In [17]:
NY511_trim.info()  #5207 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5207 entries, 1104 to 2377201
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Event Type                  5207 non-null   object 
 1   Organization Name           5207 non-null   object 
 2   Facility Name               5207 non-null   object 
 3   Direction                   5203 non-null   object 
 4   City                        4449 non-null   object 
 5   County                      5207 non-null   object 
 6   State                       5207 non-null   object 
 7   Create Time                 5207 non-null   object 
 8   Close Time                  5172 non-null   object 
 9   Event Description           5207 non-null   object 
 10  Responding Organization Id  5207 non-null   object 
 11  Latitude                    5207 non-null   float64
 12  Longitude                   5207 non-null   float64
dtypes: float64(2), object(11)
m

In [14]:
NY511_trim.head(20)

,Event Type,Organization Name,Facility Name,Direction,City,County,State,Create Time,Close Time,Event Description,Responding Organization Id,Latitude,Longitude
1104,incident,New York State Department of Transportation (R...,NY 25A,Both directions,NaN,Suffolk,NY,03/30/2020 09:00:18 PM,03/31/2020 03:48:12 AM,Downed pole on NY 25A both directions at Turke...,New York State Department of Transportation (R...,40.871400,-73.456600
1525,incident,New York State Department of Transportation (R...,NY 27,Westbound,at Babylon,Suffolk,NY,03/29/2020 11:47:24 PM,03/30/2020 01:02:11 AM,Crash on NY 27 westbound at New Highway; CR 28...,New York State Department of Transportation (R...,40.691900,-73.398800
3528,incident,New York State Department of Transportation (R...,Robert Moses Causeway,Southbound,at Islip,Suffolk,NY,03/23/2020 08:09:20 PM,03/23/2020 08:27:11 PM,Crash on Robert Moses Causeway southbound at E...,New York State Department of Transportation (R...,40.724201,-73.287002
4465,incident,New York State Department of Transportation (R...,NY 27,Westbound,NaN,Suffolk,NY,03/21/2020 02:29:14 AM,03/21/2020 04:20:13 AM,Crash on NY 27 westbound between Exit 53 - NY ...,New York State Department of Transportation (R...,40.779100,-73.013900
6696,incident,New York State Department of Transportation (R...,NY 25,Westbound,between Brookhaven,Suffolk,NY,03/17/2020 12:49:10 AM,03/17/2020 03:13:12 AM,Pedestrian accident on NY 25 westbound between...,New York State Department of Transportation (R...,40.895801,-72.883499
8972,incident,New York State Department of Transportation (R...,NY 454,Westbound,between Islip,Suffolk,NY,03/12/2020 01:06:19 PM,03/12/2020 01:27:21 PM,Brush fire on NY 454 westbound between Windsor...,New York State Department of Transportation (R...,40.793300,-73.167701
9447,incident,New York State Department of Transportation (R...,NY 454,Both directions,NaN,Suffolk,NY,03/11/2020 07:20:15 PM,03/12/2020 04:21:13 AM,Overturned vehicle on NY 454 both directions b...,New York State Department of Transportation (R...,40.801000,-73.175100
9613,incident,New York State Department of Transportation (R...,Northern State Parkway,Eastbound,at Huntington,Suffolk,NY,03/11/2020 03:44:16 PM,03/11/2020 09:39:08 PM,Police department activity on Northern State P...,New York State Department of Transportation (R...,40.823501,-73.342796
9625,incident,New York State Department of Transportation (R...,Northern State Parkway,Westbound,at Smithtown,Suffolk,NY,03/11/2020 03:25:15 PM,03/11/2020 08:53:14 PM,Police department activity on Northern State P...,New York State Department of Transportation (R...,40.818199,-73.289299
11443,incident,New York State Department of Transportation (R...,Sagtikos State Parkway,Northbound,NaN,Suffolk,NY,03/09/2020 10:36:16 AM,03/09/2020 11:11:18 AM,Crash on Sagtikos State Parkway northbound at ...,New York State Department of Transportation (R...,40.830600,-73.282200


In [18]:
#Need to exclude railroad incidents in order to maintain consistency with other road traffic datasets I have
NY511exclude = ['MTA Long Island Rail Road']
NY511_trim = NY511_trim[(~NY511_trim['Organization Name'].isin(NY511exclude))]

In [19]:
NY511_trim.info() #5207 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5207 entries, 1104 to 2377201
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Event Type                  5207 non-null   object 
 1   Organization Name           5207 non-null   object 
 2   Facility Name               5207 non-null   object 
 3   Direction                   5203 non-null   object 
 4   City                        4449 non-null   object 
 5   County                      5207 non-null   object 
 6   State                       5207 non-null   object 
 7   Create Time                 5207 non-null   object 
 8   Close Time                  5172 non-null   object 
 9   Event Description           5207 non-null   object 
 10  Responding Organization Id  5207 non-null   object 
 11  Latitude                    5207 non-null   float64
 12  Longitude                   5207 non-null   float64
dtypes: float64(2), object(11)
m

## Getting Relevant Years and Months

In [20]:
#convert times to datetimes
NY511_trim['Create Time'] = pd.to_datetime(NY511_trim['Create Time'])

In [21]:
#Periodical cicadas are confirmed in NY for Brood 5 (2016) and Brood 8 (2018) - trimmed extra years out.  Keep 2015, 2016, 2017, 2018, 2019.
NY511_trim = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2015,1,1)]
NY511_trim = NY511_trim[NY511_trim['Create Time'] < dt.datetime(2020,1,1)]

In [22]:
NY511_trim.info() #2883 entries.  Jan 2015 - Jan 2020

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2883 entries, 55058 to 1272255
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  2883 non-null   object        
 1   Organization Name           2883 non-null   object        
 2   Facility Name               2883 non-null   object        
 3   Direction                   2879 non-null   object        
 4   City                        2432 non-null   object        
 5   County                      2883 non-null   object        
 6   State                       2883 non-null   object        
 7   Create Time                 2883 non-null   datetime64[ns]
 8   Close Time                  2848 non-null   object        
 9   Event Description           2883 non-null   object        
 10  Responding Organization Id  2883 non-null   object        
 11  Latitude                    2883 non-null   float

In [13]:
#ATTEMPT 1:
#Reducing records to May and June of the years 2015, 2016, 2017, 2018, 2019.  Cicada months.


NY511_trimtest = (NY511_trim['Create Time'].dt.month == 5) & (NY511_trim['Create Time'].dt.month == 6)
NY511_trimtest.tail(60) 

# returns 'False' instead of row values...

In [134]:
#Breaking apart the datetime to date and time (don't need to include the time)
#df['new_date'] = [d.date() for d in df['my_timestamp']]
NY511_trim['Date'] = [d.date() for d in NY511_trim['Create Time']]

In [23]:
#Attempt 2:
##Reducing records to May and June of the years 2015, 2016, 2017, 2018, 2019.  Cicada months.

NY511_2015 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2015,4,30)]
NY511_2015 = NY511_2015[NY511_trim['Create Time'] < dt.datetime(2015,6,30)]

<ipython-input-23-4a3ec09bdc10>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2015 = NY511_2015[NY511_trim['Create Time'] < dt.datetime(2015,6,30)]


In [25]:
NY511_2015.info() #95 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 1208706 to 1252620
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  95 non-null     object        
 1   Organization Name           95 non-null     object        
 2   Facility Name               95 non-null     object        
 3   Direction                   95 non-null     object        
 4   City                        80 non-null     object        
 5   County                      95 non-null     object        
 6   State                       95 non-null     object        
 7   Create Time                 95 non-null     datetime64[ns]
 8   Close Time                  95 non-null     object        
 9   Event Description           95 non-null     object        
 10  Responding Organization Id  95 non-null     object        
 11  Latitude                    95 non-null     float

In [26]:
NY511_2016 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2016,4,30)]
NY511_2016 = NY511_2016[NY511_2016['Create Time'] < dt.datetime(2016,6,30)]

In [27]:
NY511_2016.info() #226 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 830101 to 921334
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  226 non-null    object        
 1   Organization Name           226 non-null    object        
 2   Facility Name               226 non-null    object        
 3   Direction                   226 non-null    object        
 4   City                        192 non-null    object        
 5   County                      226 non-null    object        
 6   State                       226 non-null    object        
 7   Create Time                 226 non-null    datetime64[ns]
 8   Close Time                  226 non-null    object        
 9   Event Description           226 non-null    object        
 10  Responding Organization Id  226 non-null    object        
 11  Latitude                    226 non-null    float6

In [28]:
NY511_2017 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2017,4,30)]
NY511_2017 = NY511_2017[NY511_trim['Create Time'] < dt.datetime(2017,6,30)]

<ipython-input-28-3ccf12137688>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2017 = NY511_2017[NY511_trim['Create Time'] < dt.datetime(2017,6,30)]


In [29]:
NY511_2017.info() #93 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 574438 to 619707
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  93 non-null     object        
 1   Organization Name           93 non-null     object        
 2   Facility Name               93 non-null     object        
 3   Direction                   93 non-null     object        
 4   City                        79 non-null     object        
 5   County                      93 non-null     object        
 6   State                       93 non-null     object        
 7   Create Time                 93 non-null     datetime64[ns]
 8   Close Time                  93 non-null     object        
 9   Event Description           93 non-null     object        
 10  Responding Organization Id  93 non-null     object        
 11  Latitude                    93 non-null     float64

In [30]:
NY511_2018 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2018,4,30)]
NY511_2018 = NY511_2018[NY511_trim['Create Time'] < dt.datetime(2018,6,30)]

<ipython-input-30-c0fb109576af>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2018 = NY511_2018[NY511_trim['Create Time'] < dt.datetime(2018,6,30)]


In [31]:
NY511_2018.info()  #130 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 321431 to 368526
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  130 non-null    object        
 1   Organization Name           130 non-null    object        
 2   Facility Name               130 non-null    object        
 3   Direction                   130 non-null    object        
 4   City                        107 non-null    object        
 5   County                      130 non-null    object        
 6   State                       130 non-null    object        
 7   Create Time                 130 non-null    datetime64[ns]
 8   Close Time                  130 non-null    object        
 9   Event Description           130 non-null    object        
 10  Responding Organization Id  130 non-null    object        
 11  Latitude                    130 non-null    float6

In [32]:
NY511_2019 = NY511_trim[NY511_trim['Create Time'] > dt.datetime(2019,4,30)]
NY511_2019 = NY511_2019[NY511_trim['Create Time'] < dt.datetime(2019,6,30)]

<ipython-input-32-03f28f370c15>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NY511_2019 = NY511_2019[NY511_trim['Create Time'] < dt.datetime(2019,6,30)]


In [105]:
NY511_2019.info() #0 entries.  There might be a gap in the data. See below

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  0 non-null      object        
 1   Organization Name           0 non-null      object        
 2   Facility Name               0 non-null      object        
 3   Direction                   0 non-null      object        
 4   City                        0 non-null      object        
 5   County                      0 non-null      object        
 6   State                       0 non-null      object        
 7   Create Time                 0 non-null      datetime64[ns]
 8   Close Time                  0 non-null      object        
 9   Event Description           0 non-null      object        
 10  Responding Organization Id  0 non-null      object        
 11  Latitude                    0 non-null      float64       
 12  Longit

In [1]:
#Why no rows for 2019?
NY511_trim.sort_values('Create Time', ascending=False)
NY511_trim.head(1200)
#No rows between 2019-07-01 00:11:15 and 2019-04-03 07:56:17.  Data in data collection? Bug from my trimming?

NameError: name 'NY511_trim' is not defined

---------------------------------

--------------------------

## Export the data

In [33]:
## Bringing the yearly dataframes into a single dataframe

years = [NY511_2015, NY511_2016, NY511_2017, NY511_2018, NY511_2019]
NY511_final = pd.concat(years)

In [34]:
#Create a column specifically for the date

NY511_final['Date'] = [d.date() for d in NY511_final['Create Time']]
NY511_final['Date'] = pd.to_datetime(NY511_final['Date'])

In [36]:
NY511_final.info() #544 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 1208706 to 368526
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Event Type                  544 non-null    object        
 1   Organization Name           544 non-null    object        
 2   Facility Name               544 non-null    object        
 3   Direction                   544 non-null    object        
 4   City                        458 non-null    object        
 5   County                      544 non-null    object        
 6   State                       544 non-null    object        
 7   Create Time                 544 non-null    datetime64[ns]
 8   Close Time                  544 non-null    object        
 9   Event Description           544 non-null    object        
 10  Responding Organization Id  544 non-null    object        
 11  Latitude                    544 non-null    float

In [37]:
#Save new dataframe as a csv
NY511_final.to_csv('NY511_final.csv', index = False, header=True)

In [38]:
##Just curious now!
NY511_final

,Event Type,Organization Name,Facility Name,Direction,City,County,State,Create Time,Close Time,Event Description,Responding Organization Id,Latitude,Longitude,Date
1208706,incident,New York State Department of Transportation (R...,NY 27,Westbound,NaN,Suffolk,NY,2015-06-28 03:01:00,06/28/2015 06:00:00 AM,Emergency construction on NY 27 westbound betw...,New York State Department of Transportation (R...,40.742100,-73.237600,2015-06-28
1208830,accident,New York State Department of Transportation (R...,I-495,Eastbound,between Islip,Suffolk,NY,2015-06-27 11:55:00,06/27/2015 12:39:00 PM,Accident on I-495 eastbound between Exit 56 - ...,New York State Department of Transportation (R...,40.806999,-73.231399,2015-06-27
1209146,incident,New York State Department of Transportation (R...,Southern State Parkway,Eastbound,at Islip,Suffolk,NY,2015-06-27 02:20:00,06/27/2015 03:22:00 AM,Accident on Southern State Parkway eastbound a...,New York State Department of Transportation (R...,40.756900,-73.194200,2015-06-27
1209453,accident,New York State Department of Transportation (R...,I-495,Westbound,between Islip,Suffolk,NY,2015-06-26 09:15:00,06/26/2015 09:38:00 AM,Accident on I-495 westbound between Exit 55 - ...,New York State Department of Transportation (R...,40.805900,-73.241401,2015-06-26
1210184,incident,New York State Department of Transportation (R...,I-495,Westbound,at Islip,Suffolk,NY,2015-06-25 11:13:00,06/25/2015 11:24:00 AM,Accident on I-495 service road westbound at Ha...,New York State Department of Transportation (R...,40.808700,-73.167396,2015-06-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366865,accident,New York State Department of Transportation (R...,I-495,Westbound,between Huntington,Suffolk,NY,2018-05-01 16:46:17,05/01/2018 05:54:24 PM,Accident on I-495 westbound between Exit 49S -...,New York State Department of Transportation (R...,40.782501,-73.433998,2018-05-01
366960,accident,New York State Department of Transportation (R...,Northern State Parkway,Eastbound,at Smithtown,Suffolk,NY,2018-05-01 15:22:17,05/01/2018 04:58:18 PM,Accident on Northern State Parkway eastbound a...,New York State Department of Transportation (R...,40.818500,-73.293500,2018-05-01
366979,accident,New York State Department of Transportation (R...,I-495,Eastbound,at Islip,Suffolk,NY,2018-05-01 15:05:18,05/01/2018 04:23:19 PM,Accident on I-495 eastbound at Exit 56 - Wheel...,New York State Department of Transportation (R...,40.806999,-73.231399,2018-05-01
367002,accident,New York State Department of Transportation (R...,Southern State Parkway,Eastbound,at Babylon,Suffolk,NY,2018-05-01 14:51:20,05/01/2018 04:33:26 PM,Accident on Southern State Parkway eastbound a...,New York State Department of Transportation (R...,40.731998,-73.320297,2018-05-01
